# UKHSA: COVID-19 confirmed cases over time (England)

## Goal
Plot **time vs infected** using a straightforward proxy: **confirmed COVID-19 cases by day**.

## Data provenance
- Source: UKHSA dashboard API (`https://ukhsa-dashboard.data.gov.uk/access-our-data`)
- Metric: `COVID-19_cases_casesByDay` (England)
- Log the download date + metric details in `../docs/DATA_SOURCES.md`


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from epidemiology_project.paths import raw_data_dir, figures_dir
from epidemiology_project.ukhsa import UksHaMetricQuery, fetch_metric


## Download (UKHSA API)


In [ ]:
q = UksHaMetricQuery(
    theme='infectious_disease',
    sub_theme='respiratory',
    topic='COVID-19',
    geography_type='Nation',
    geography='England',
    metric='COVID-19_cases_casesByDay',
)

df = fetch_metric(q)
df[['date','metric_value']].head()


## Save raw extract (recommended)


In [ ]:
out = raw_data_dir() / 'ukhsa_covid19_confirmed_cases_by_day_england.csv'
df.to_csv(out, index=False)
out


## Plot: time vs infected (confirmed cases)


In [ ]:
series = df[['date', 'metric_value']].dropna().sort_values('date')
series = series.rename(columns={'metric_value': 'cases'})

fig, ax = plt.subplots(figsize=(11, 5))
ax.plot(series['date'], series['cases'], linewidth=1)
ax.set_title('England: confirmed COVID-19 cases by day (UKHSA)')
ax.set_xlabel('Date')
ax.set_ylabel('Cases')
ax.grid(True, alpha=0.3)
fig.tight_layout()

fig_path = figures_dir(final=True) / 'ukhsa_covid19_cases_by_day_england.png'
fig.savefig(fig_path, dpi=200)
fig_path


## Optional: 7-day rolling mean (smoother)


In [ ]:
series2 = series.set_index('date')
series2['cases_7d_mean'] = series2['cases'].rolling(7).mean()

fig, ax = plt.subplots(figsize=(11, 5))
ax.plot(series2.index, series2['cases_7d_mean'], linewidth=1.5)
ax.set_title('England: confirmed COVID-19 cases (7-day rolling mean)')
ax.set_xlabel('Date')
ax.set_ylabel('Cases (7-day mean)')
ax.grid(True, alpha=0.3)
fig.tight_layout()

fig_path = figures_dir(final=True) / 'ukhsa_covid19_cases_by_day_england_7dmean.png'
fig.savefig(fig_path, dpi=200)
fig_path
